In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from astroquery.gaia import Gaia
import astropy.units as u
from astropy.coordinates import SkyCoord

Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source"  # Reselect Data Release 3, default
Gaia.ROW_LIMIT = -1

In [ ]:
pixel_scale = 0.11 * u.arcsec / u.pix
n_pixels = 4000 * u.pix
sca_side = n_pixels * pixel_scale

In [ ]:
latitudes = np.linspace(0, 89, 5)
coords = SkyCoord([SkyCoord(l=5, b=b, unit=u.deg, frame='galactic') for b in latitudes])

In [ ]:
columns = ['source_id', 'ra', 'dec', 'phot_g_mean_mag', 'phot_rp_mean_mag']

queries = [
    Gaia.query_object(
        coord, width=sca_side, height=sca_side, columns=columns
    ) for coord in coords
]

In [ ]:
n_sources = np.array([len(q) for q in queries])
plt.semilogy(latitudes, n_sources, 'o-k', label='Single SCA')
plt.semilogy(latitudes, 18 * n_sources, 'o-r', label='18 SCAs')
plt.gca().set(
    xlabel='Galactic latitude (deg)',
    ylabel='Gaia sources'
)
plt.legend()
plt.savefig('plots/n_sources_with_latitude.png', dpi=200, bbox_inches='tight')

In [ ]:
center = queries[0]
pole = queries[-1]

n, edges = plt.hist((center['phot_g_mean_mag']), alpha=0.5, log=True, label='Galactic center')[:2]
plt.hist((pole['phot_g_mean_mag']), alpha=0.5, label='Galactic pole')

edges = np.array(edges)
x = 0.5 * (edges[1:] + edges[:-1])

roman_mags = np.linspace(18, 26, 100)
fit = np.polyfit(x, np.log10(n), 1)

roman_center_sources = 10**np.polyval(fit, roman_mags)
ratio_pole_center = len(pole) / len(center)
roman_pole_sources = ratio_pole_center * 10**np.polyval(fit, roman_mags)

plt.plot(roman_mags, roman_center_sources, ls='--', color='C0', lw=2, label='Roman, center (extrapolated)')

sources_in_gaia_and_roman = (
    (roman_mags < center['phot_g_mean_mag'].max()) & 
    (roman_mags > 18)
)
n_sources_center_roman = np.trapz(10**np.polyval(fit, roman_mags), roman_mags)
n_sources_center_roman_gaia = np.count_nonzero(center['phot_g_mean_mag'] > 18)

plt.plot(roman_mags, roman_pole_sources, ls='--', color='C1', lw=2, label='Roman, pole (Extrapolated)')
plt.legend()
plt.gca().set(
    xlabel='Gaia G [mag]',
    ylabel='Sources per SCA'
)
plt.savefig('plots/n_sources.png', bbox_inches='tight', dpi=200)
print(f'N sources per SCA in Gaia (center) = {len(center):.0f}') 
print(f'N sources per SCA in Gaia (pole) = {len(pole):.0f}') 

print(f'N sources per SCA in Roman (center) = {n_sources_center_roman:.0f}') 
print(f'N sources per SCA in Roman (pole) = {ratio_pole_center * n_sources_center_roman:.0f}') 

print(f'N sources per SCA in both Roman and Gaia (center) = {n_sources_center_roman_gaia:.0f}') 
print(f'N sources per SCA in both Roman and Gaia (pole) = {ratio_pole_center * n_sources_center_roman_gaia:.0f}') 